# Transformer 실습


## 1. 라이브러리 준비

In [20]:
!pip install datasets sacremoses

## 2. 데이터셋 준비

In [21]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


# ds = load_dataset("stanfordnlp/imdb")
train_ds = load_dataset("stanfordnlp/imdb", split="train")
test_ds = load_dataset("stanfordnlp/imdb", split="test")

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
  max_len = 400
  texts, labels = [], []

  # 배치 데이터의 각 행마다 반복
  for row in batch:
    # truncation이 True -> max length가 넘어가면 자름. 끝에서 3번째 토큰을 label(정답)로 선택
    labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3])
    # 마지막 3개를 제외한 앞 부분의 토큰들을 텐서로 변환한 후 텍스트(입력)로 사용
    texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-3]))

  # 패딩 토큰 추가 (아마 400자 기준이겠지?)
  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
  # labels를 텐서로 변환
  labels = torch.LongTensor(labels)

  return texts, labels

# 각각 트레인 로더, 테스트 로더를 만들어준다. 배치사이즈는 64
train_loader = DataLoader(
    train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


- `tokenizer` → BERT 모델(bert-base-uncased)의 토크나이저를 가져옴
    - bert : BERT 모델
    - base : 12층 레이어, 은닉층 사이즈 768
    - uncased  : 대소문자 구분 없이, 소문자로 바꿔서 처리함

- 데이터의 배치 사이즈 64개 → 64개씩 묶어서 진행
- label 값 = text 값(=리뷰 내용)의 토큰들 중 끝에서 3번째
    - 왜 뒤에서 3번째일까…? 문장부호에 해당하는 토큰이나 `[SEP]` 토큰을 거르기 위해서인 것 같다.
    - 아무래도 아래와 같이 토크나이즈 되는 경우가 많을 거라 생각된다.

    ```['this', 'film', 'does', "n't", 'have', 'much', 'of', 'a', 'plot', '.', '[SEP]']```
- 입력 값 = 처음부터 끝에서 3번째까지의 값

### input 차원 체크

In [50]:
text, label = next(iter(train_loader))
print(text.shape, label.shape)

torch.Size([64, 397]) torch.Size([64])


## Self-attention


Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [22]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    # 입력 벡터 차원
    self.input_dim = input_dim

    # Q, K, V 및 최종 출력 차원
    self.d_model = d_model

    # Q, K, V에 곱할 가중치 행렬값
    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)

    # 은닉층 한 번 선형 결과를 만들어서 정확도를 더 높여준다
    self.dense = nn.Linear(d_model, d_model)

    # 어텐션 스코어 계산에 필요한 소프트맥스. 확률 분포로 변환시켜준다. 0~1사이의 값
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    # Q, K, V 벡터 값 생성
    q, k, v = self.wq(x), self.wk(x), self.wv(x) # (B, S, D)
    # Q * K^T 쿼리와 키의 유사도를 구함
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    # 스케일링 : 루트 d로 나누어서 크기 조정을 한다.
    score = score / sqrt(self.d_model)

    # 마스킹 - 패딩 토큰 무시. 0에 수렴하도록? 거의 안보이게 하기 위해 -1e9 더함
    if mask is not None:
      score = score + (mask * -1e9)

    # 스코어를 소프트 맥스로 변환
    score = self.softmax(score)
    # 어텐션 스코어를 value의 벡터에 곱해서 문맥을 반영한 최종 벡터 꺼냄
    result = torch.matmul(score, v) # -> (B, S, D)
    # 최종 출력값 변환
    result = self.dense(result) # -> (B, S, D)

    return result

- 마스크에 대해 의문인 점 🙋‍♀️
    - mask의 shape가 (B, S, 1)라고 써져있는데
    - TextClassifier 클래스에서 가운데가 None인데 (B, S, 1) 아닌가..?
    ```    
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    ```

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

## 4. 트랜스포머 레이어 정의

In [24]:
#트랜스포머
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim # 입력 벡터 차원
    self.d_model = d_model # 모델 크기
    self.dff = dff #은닉층 크기

    self.sa = SelfAttention(input_dim, d_model) #어텐션 계산

    # 그냥 우리가 알고있는 MLP
    # 언어를 학습시키는 데 더 많은 가중치(weight)를 부여함
    # 비선형 처리 + 추가 가중치
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask) #1. 셀프 어텐션의 결과를 가져온다.
    x = self.ffn(x) #2. 어텐션 결과를 FFN(MLP)에 통과시킨다

    return x

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [34]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates
# 포지셔널 인코딩
# 트랜스포머를 사용하려면 순서 정보 벡터값을 따로 계산해서 입력해줘야한다.
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다

.... 이 부분 어려워서 패스하겠습니다 😞

In [35]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    # 워드 임베딩
    self.embedding = nn.Embedding(vocab_size, d_model)
    # 포지셔널 인코딩
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    # rnn 대신 트랜스포머 레이어를 통과시킴.
    # 인코더, 디코더를 여러개 두고 쓰는 방식이라서 여러개를 만들어서 통과시키는듯
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    # 마지막 토큰을 예측 -> 학습 데이터셋들의 모든 토큰들의 개수를 체크하는게 맞겠지 여기서 가장 유리한 것을 골라야하니까
    self.classification = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    # x의 shape == (batch_size, seq_len, d_model)
    # 마스크 정의: 패딩 토큰이 있다면 true
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :] # -> (B,1,S)?

    # 시퀀스 데이터의 길이
    seq_len = x.shape[1]

    # 워드 임베딩
    x = self.embedding(x)
    # 노멀라이즈?
    x = x * sqrt(self.d_model)
    # 포지셔널 인코딩 - 위치 정보 벡터 추가
    x = x + self.pos_encoding[:, :seq_len]

    # 트랜스포머의 레이어 갯수만큼 돌려라...
    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0] # 첫번째 차원(배치)에서 모든 요소를 선택, 두번째 차원(시퀀스 데이터) 첫번째 요소만 선택해서 마지막 단어를 예측하게끔 하는듯
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

In [38]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
# 교차 엔트로피로 수정
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

교차 엔트로피 손실 함수를 쓰는 이유?

- 마지막 단어 예측 → 데이터셋의 모든 단어들 중 하나일 가능성일 확률이 높음
- TextClassifier 클래스의 최종 출력 값의 shape vocab_size과 동일함
    - 즉, MNIST 분류 문제처럼 모든 토큰들 중 하나를 분류하는 것.
    - 교차 엔트로피는 분류 문제에서 예측값들의 정답 확률 분포를 알려줌
    - `argmax(logits, dim=-1)` 을 통해 가장 높은 확률을 선택

In [32]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # 가장 큰 값이 곧 가장 정답에 가까운 예측값
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [39]:

n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    # 수많은 값을 예측해서 가장 큰 값을 고르는게 포인트
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 3021.578178882599
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   1 | Train Loss: 2695.5272641181946
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   2 | Train Loss: 2666.333320617676
=========> Train acc: 0.036 | Test acc: 0.034
Epoch   3 | Train Loss: 2655.3203144073486
=========> Train acc: 0.039 | Test acc: 0.039
Epoch   4 | Train Loss: 2649.541160106659
=========> Train acc: 0.042 | Test acc: 0.041
Epoch   5 | Train Loss: 2644.4300560951233
=========> Train acc: 0.041 | Test acc: 0.038
Epoch   6 | Train Loss: 2638.7094736099243
=========> Train acc: 0.041 | Test acc: 0.041
Epoch   7 | Train Loss: 2624.6238555908203
=========> Train acc: 0.040 | Test acc: 0.040
Epoch   8 | Train Loss: 2607.0720953941345
=========> Train acc: 0.042 | Test acc: 0.039
Epoch   9 | Train Loss: 2591.545946121216
=========> Train acc: 0.041 | Test acc: 0.037
Epoch  10 | Train Loss: 2576.376081466675
=========> Train acc: 0.044 | Test acc: 0.041
Epoch  11 | Train Loss: 25

정확도가 0.1 대인데.. 제대로 학습된 거 맞나요🥺

In [69]:
from torch.nn.functional import softmax

# 1. 테스트 샘플 1개 가져오기
sample = test_ds[18]['text']
tokens = tokenizer(sample, truncation=True, max_length=400).input_ids

# 2. 정답은 뒤에서 -3 인덱스인 토큰, 입력은 그 앞부분
target_ids = tokens[-3]         # 정답
input_ids = tokens[:-3]          # 입력

# 3. 텐서로 변환해서 배치처럼 만들기
input_tensor = torch.LongTensor([input_ids]).to(device)  # shape: (1, L)

# 4. 모델에 넣기
model.eval()
with torch.no_grad():
    logits = model(input_tensor)
    pred_ids = logits.argmax(dim=-1)[0]

# 5. 예측 결과 디코딩
pred_tokens = tokenizer.decode(pred_ids.tolist()) #토큰 ID -> 문자
target_tokens = tokenizer.decode(target_ids)

print("🟦 전체 입력 문장:", sample)
print("🟦 입력 문장:", tokenizer.decode(input_ids))
print("🟩 정답 토큰:", target_tokens)
print("🟥 예측 토큰:", pred_tokens)


🟦 전체 입력 문장: Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a summer job as an assistant to a foul-mouthed, eccentric, once-famous and now-forgotten actress Evie Walton, (Julie Walters), that he finally finds himself in true 'Harold and Maude' fashion. Of course, Evie is deeply unhappy herself and it's only when these two sad sacks find each other that they can put their mutual misery aside and hit the road to happiness.<br /><br />Of course it's corny and sentimental and very predictable but it has a hard side to it, too and Walters, who could sleep-walk her way through this sort of thing if she wanted, is excellent. It's when she puts the craziness to one side and finds the pathos in the character, (like hitting the bottle and throwing up in the sink), that sh